## Lisamaterjal: _PrettyPrinter_ ja märgenduskihtide visualiseerimine 

Mõnikord on tarvis saada märgendustest kiire ülevaade. Selleks oleks tore kuvada tekst selliselt, et märgendused on kuidagi spetsiaalselt tähistatud või esile tõstetud. EstNLTK-s on selleks puhuks loodud klass [PrettyPrinter](https://estnltk.github.io/estnltk/1.4.1/tutorials/prettyprinter.html#html-prettyprinter), mis võimaldab märgenduskihte visualiseerida. Otseselt pilte see klass küll ei joonista, aga ta vormistab teksti HTML-i ehk veebilehe lähtekoodina, mida saab edaspidi juba mitmesugustel viisidel kuvada.

Näide _PrettyPrinter_-i kasutamise kohta:

In [1]:
from estnltk import Text
from estnltk import PrettyPrinter

text = Text('''" Kas sa leiutad mingi riistapuu , millega saaks maja paarsada meetrit eemale vedada ? "
               " Selliseid riistu pole vaja leiutada , need on juba olemas . "''')
text.tag_verb_chains()

# Loome uue "kaunistrükkija", 
#     mis värvib kihi 'verb_chains' taustavärviga 'cyan' (sinine/tsüaniid)
pp = PrettyPrinter( background='verb_chains', background_value='cyan' )

# vormistame teksti HTML kujul
html_source = pp.render(text, True)
html_source

'<!DOCTYPE html>\n<html>\n    <head>\n        <link rel="stylesheet" type="text/css" href="prettyprinter.css">\n        <meta charset="utf-8">\n        <title>PrettyPrinter</title>\n    </head>\n    <style>\n\n\n\t\tmark {\n\t\t\tbackground:none;\n\t\t}\n\t\tmark.background {\n\t\t\tbackground-color: cyan;\n\t\t}\n\n    </style>\n    <body>\n\n&quot; Kas sa <mark class="background">leiutad</mark> mingi riistapuu , millega <mark class="background">saaks</mark> maja paarsada meetrit eemale <mark class="background">vedada</mark> ? &quot;<br/>               &quot; Selliseid riistu <mark class="background">pole</mark> <mark class="background">vaja</mark> <mark class="background">leiutada</mark> , need <mark class="background">on</mark> juba <mark class="background">olemas</mark> . &quot;\n\t</body>\n</html>'

Üks viis näha vormistatud teksti on salvestada nüüd `html_source` sisu veebilehekülje faili (ehk `.html` lõpuga tekstifaili) ning avada see fail veebibrauseris. Võid järgi katsetada, kuidas see töötab.

Aga _Notebook_ oskab ka ise HTML sisu kuvada. Seda saab teha funktsioonide `HTML` ja `display` abil:

In [2]:
from IPython.display import HTML, display
# kuvame HTML sisu Notebook'is
display(HTML(html_source))

Detailsema visualiseerimisvõimaluste kirjelduse leiate [PrettyPrinter-i abimaterjalist](https://estnltk.github.io/estnltk/1.4.1/tutorials/prettyprinter.html#html-prettyprinter). 
Paraku ei ole _PrettyPrinter_-i implementatsioon versioonis 1.4 perfektne: mõned asjad on abimaterjalis ebatäpsed ning mõned asjad ei tööta ka moodulis endas nii, nagu intuitiivselt võiks oodata. 
Järgnevalt juhime neile kitsaskohtadele tähelepanu ning tutvustame ka võimalikke lahendusi.

**Esimene nüanss: nõuded märgenduskihile.** _PrettyPrinter_ töötab ainult selliste märgenduskihtide peal, kus iga üksikmärgendus on sõnastiku kujul ning sõnastiku atribuutides `'start'` ja `'end'` on toodud märgenduse asukoht ning atribuudis `'text'` vastav tekstisisu. Seetõttu ei tööta ka [abimaterjalis](https://estnltk.github.io/estnltk/1.4.1/tutorials/prettyprinter.html#html-prettyprinter) olev sõnaliikide (POS-tagide) värvimise näide (Example #3) -- sõnaliikide kiht pole eraldiseisev kiht, vaid on morf analüüsi kihi sees ja nii sügavale _PrettyPrinter_ ei vaata.  

Selleks, et sõnaliikide järgi värvimine korrektselt töötaks, tuleb värvitava sõnaliigi (või sõnaliikide) kohta tekitada _Text_ objekti sisse uus kiht, mille liikmed on nõutud kujul. Lihtne näide:

In [3]:
text = Text('''Kas sa leiutad mingi riistapuu?''')
text.tag_analysis()

# Tekitame eraldiseisva verbide märgenduskihi
text['verbs'] = [ {'end': 14, 'start': 7, 'text': 'leiutad'} ]

# vormistame teksti HTML kujul, märgendame 'verbs' kihi
pp = PrettyPrinter( background='verbs', background_value='cyan' )
html_source = pp.render(text, True)

# kuvame
display(HTML(html_source))

**Teine nüanss: erinvate vorminguelementide ja märgenduskihtide kombineerimine.** 
Ühe vorminguelemendi (stiili) saab siduda ainult ühe konkreetse märgenduskihiga.
Isegi taustavärvi puhul on nii, et seda saab kasutada ainult ühe kihi jaoks, aga ei saa omistada erinevatele kihtidele erinevaid värve.
Ei saa näiteks teha nii, et osalausete taustavärv on kollane ja nende sees olevate verbiahelate taustavärv on  roheline.
Küll aga saab teha nii, et osalausete taustavärv on kollane ja nende sees olevad verbiahelatesse kuuluvad sõnad on märgitud rohelist värvi tähtedega (kuna taustavärv ja tähevärv on erinevad vorminguelemendid).
Näide:

In [4]:
# Loome teksti ja lisame osalausete ja verbiahelate märgendused
from estnltk import Text
text = Text('Küll homme armastab see, kes ei armastanud eile.')
text = text.tag_verb_chains()

In [5]:
from estnltk import PrettyPrinter
from IPython.display import HTML, display

# Loome uue "kaunistrükkija", 
#     mis värvib kihi 'clauses' taustavärviga '#FFFFA0' (helekollane)
#     ning värvib tähed kihis 'verb_chains' värviga 'green' (roheline)
pp = PrettyPrinter( background='clauses', background_value='#FFFFA0', color='verb_chains', color_value='green' )
html_source = pp.render(text, True)

# kuvame
display(HTML(html_source))

**-- _Aga kui ma tahan siiski kasutada erinevaid värve erinevate sõnade/fraaside vms jaoks. Kuidas seda teha?_ **
 
Kui eesmärgiks on värvida kindla (tausta-)värviga kõik sõnad tekstis, hoolimata nende asukohast, siis saab teha nagu [abimaterjalis](https://estnltk.github.io/estnltk/1.4.1/tutorials/prettyprinter.html#html-prettyprinter) õpetatakse: defineerida reeglid, kus igale sõnale on seatud vastavusse mingi värv ning rakendada reegleid `'words'` kihi esiletoomisel taustavärviga. Näide:

In [6]:
text = Text('See on, nagu on, ja teisiti ei ole').tag_analysis()
rules =[
            ('on', '#00ccff'),
            ('ei', '#ff6666'),
            ('ole', '#ff6666'),
        ]
pp = PrettyPrinter(background='words', background_value=rules)
html = pp.render(text, True)
display(HTML(html))

Analoogselt toimib see ka fraaside korral: reeglites tuleb lihtsalt täpselt kirjeldada, millised fraasid millist värvi värvitakse:

In [11]:
text = Text('Ooperitäht José Carreras saabus eralennukiga Tallinna').tag_named_entities()
rules =[
            ('José Carreras', '#00ccff'),
            ('Tallinna', '#ff6666'),
        ]
pp = PrettyPrinter(background='named_entities', background_value=rules)
html = pp.render(text, True)
display(HTML(html))

Selle lähenemise miinused. 
Esiteks, värvimine on tõstutundlik, seega tuleks iga sõna puhul reeglites täpsustada kõik võimalikud suur-väiketäht varieeruvused (nt _'on'_, _'On'_, _'ON'_, _'oN'_).
Teiseks, värvitakse kõik leitud sõnad / fraasid, seega ei sobi lähenemine siis, kui tahame värvida valikuliselt: mõnes kontekstis värvida ja mõnes jätta värvimata.

**_-- Kuidas värvida sõnu / fraase valikuliselt?_**

_PrettyPrinter_-i abil paraku ei saagi. Küll aga saad selle lahenduse ise välja nuputada (midagi analoogset oled juba ka teinud  5. praksis). 
Sisuliselt on vaja: A) värvitavate tekstiüksuste koordinaate tekstis (mille leiab märgendusele vastava sõnastiku  atribuutidest `'start'` ja `'end'`), B) sisendteksti sõne kujul, C) mõningasi teadmisi sellest, kuidas HTML-is teksti kujundada (vt nt [siit](https://www.w3schools.com/html/html_styles.asp) ja [siit](https://www.w3schools.com/css/css_text.asp) ). 
Sisendtekst (B) tuleb ümber kirjutada nii, et värvitavad tekstiüksused (A) ümbritsetakse tekstivormingut/värvi kirjeldavate HTML märgenditega (C) ja tulemuseks on HTML lähtekood, mida saab kuvada kas funktsioonide `HTML` ja `display` abil või panna HTML faili ja vaadata veebibrauseris.